In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

In [3]:
train_path = "train_telugu_news.csv"

telugu_news_df = pd.read_csv(train_path)
telugu_news_df

,SNo,date,heading,body,topic
0,414,11-05-2017 00:39:13,ఐడిబిఐపై ఆర్‌బిఐ నజర్‌,భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ ...,business
1,2668,12-06-2017 00:40:32,బ్యాంకింగ్‌ చీఫ్‌లతో నేడు జైట్లీ భేటీ,న్యూఢిల్లీ : ఆర్థిక మంత్రి అరుణ్‌ జైట్లీ సోమవా...,business
2,19923,19-01-2017 19:51:31,కీలక వికెట్ తీసిన జడేజా..,కటక్: ఇంగ్లండ్‌తో జరుగుతున్న సెకండ్ వన్డే మ్యా...,sports
3,15762,23-11-2017 17:29:04,మరో రెచ్చగొట్టే చర్యకు దిగిన పాకిస్థాన్,\nఇస్లామాబాద్ : పాకిస్థాన్ అంతర్జాతీయ ఉగ్రవాది...,nation
4,8624,03-04-2017 15:48:23,గోవాలో కొడుకుతో కలిసి అల్లు అర్జున్ స్విమ్మింగ్!,స్టార్‌ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటి...,entertainment
...,...,...,...,...,...
17307,10296,02-02-2017 12:18:11,"చిరు,ఎన్టీఆర్ వల్ల కాజల్‌కు దక్కిందేమిటి..?",ఇండస్ట్రీలో హీరోయిన్‌గా రాణించాలంటే కేవలం గ్లా...,entertainment
17308,13641,20-12-2017 04:16:32,"హిమాచల్‌ సీఎం రేసులో నడ్డా, జైరామ్‌","షిమ్లా, డిసెంబరు 19: హిమాచల్‌ ప్రదేశ్‌ అసెంబ్ల...",nation
17309,17288,20-05-2017 03:00:10,సతతహరిత విప్లవం అవసరం,2022 నాటికి రైతుల ఆదాయం రెట్టింపు చేయడానికి క...,nation
17310,20882,13-06-2017 04:30:36,ఇంగ్లండ్‌ జెర్సీలో వార్న్‌..!,పందెంలో ఓడిన షేనలండన్: ఆస్ట్రేలియా స్పిన్ లెజె...,sports


In [4]:
del telugu_news_df["heading"]
del telugu_news_df["date"]

In [5]:
import re
from indicnlp.tokenize import sentence_tokenize, indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

def preprocess_text(text):
    sentences = sentence_tokenize.sentence_split(text, lang='te')
    tokenizer = indic_tokenize.trivial_tokenize

    preprocessed_sentences = []
    for sentence in sentences:
        sentence = re.sub('[^\u0C00-\u0C7F]+', ' ', sentence)
        words = tokenizer(sentence)
        filtered_words = [word for word in words if word.strip() and not re.match(r'^[.,!?]+$|^["\'()\[\]]+$', word)]
        preprocessed_sentence = ' '.join(filtered_words)
        preprocessed_sentences.append(preprocessed_sentence)

    preprocessed_text = ' '.join(preprocessed_sentences)
    return preprocessed_text

In [6]:
telugu_news_df['body'] = telugu_news_df['body'].apply(preprocess_text)
print(telugu_news_df.head())

     SNo                                               body          topic
0    414  భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ ...       business
1   2668  న్యూఢిల్లీ ఆర్థిక మంత్రి అరుణ్ జైట్లీ సోమవారం ...       business
2  19923  కటక్ ఇంగ్లండ్ తో జరుగుతున్న సెకండ్ వన్డే మ్యాచ...         sports
3  15762  ఇస్లామాబాద్ పాకిస్థాన్ అంతర్జాతీయ ఉగ్రవాది హపీ...         nation
4   8624  స్టార్ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటిక...  entertainment


In [7]:
topic_mapping = {'business': 0, 'sports': 1, 'nation': 2, 'entertainment': 3, 'editorial': 4}
telugu_news_df['topic'] = telugu_news_df['topic'].replace(topic_mapping)
telugu_news_df["topic"].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [8]:
text_data = telugu_news_df['body'].tolist()
labels = telugu_news_df['topic'].tolist()

In [9]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
input_ids = []
attention_masks = []

In [10]:
for text in text_data:
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])

In [11]:
input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)
labels = tf.convert_to_tensor(labels)

# Split the data into train and test sets
train_size = int(0.8 * len(input_ids))
train_input_ids, test_input_ids = input_ids[:train_size], input_ids[train_size:]
train_attention_masks, test_attention_masks = attention_masks[:train_size], attention_masks[train_size:]
train_labels, test_labels = labels[:train_size], labels[train_size:]

In [12]:
# Define the BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=5)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model.summary()


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 177,857,285
Trainable params: 177,857,285
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(
    x=[train_input_ids, train_attention_masks],
    y=train_labels,
    epochs=4,
    batch_size=32,
    validation_data=([test_input_ids, test_attention_masks], test_labels)
)

Epoch 1/4
 12/433 [..............................] - ETA: 2:19:46 - loss: 1.5673 - accuracy: 0.3099

In [ ]:
new_texts = ["ఈ చిత్రం చాలా మంచిది", "నేను ఈ పుస్తకాన్ని చదవగలను"]
new_input_ids = []
new_attention_masks = []

for text in new_texts:
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    new_input_ids.append(encoded['input_ids'])
    new_attention_masks.append(encoded['attention_mask'])

new_input_ids = tf.concat(new_input_ids, axis=0)
new_attention_masks = tf.concat(new_attention_masks, axis=0)

predictions = model.predict([new_input_ids, new_attention_masks])
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

for text, label in zip(new_texts, predicted_labels):
    print("Text:", text)
    print("Predicted Label:", label)